In [181]:
# imports + see where I'm at

import pandas as pd
import os
from datetime import timedelta, date
from functools import reduce

os.getcwd()


'/home/mike/Documents/dev/2021_10_DSI_WD/mikezambeck/python/tms/notebooks'

In [182]:
# Load data. make dates dates, id columns

# M2
dfM2 = pd.read_csv('../data_trunc/M2.csv', header=None, skiprows=[0],
                   parse_dates=[0], names=['DATE','M2'])

# travelers checks
dfTC = pd.read_csv('../data_trunc/WTCSL.csv', header=None, skiprows=[0],
                   parse_dates=[0], names=['DATE','TC'])

# large time deposits
dfLTD = pd.read_csv('../data_trunc/LTDACBM027NBOG.csv', header=None, skiprows=[0],
                   parse_dates=[0], names=['DATE','LTD'])

# small time deposits
dfSTD = pd.read_csv('../data_trunc/WSMTIME.csv', header=None, skiprows=[0],
                   parse_dates=[0], names=['DATE','STD'])

# retail money market funds
dfMMF = pd.read_csv('../data_trunc/WRMFSL.csv', header=None, skiprows=[0],
                   parse_dates=[0], names=['DATE','RMMF'])

# treasury deposits
dfTD = pd.read_csv('../data_trunc/WTREGEN.csv', header=None, skiprows=[0],
                   parse_dates=[0], names=['DATE','TD'])



In [183]:
dfM2.head(5)

,DATE,M2
0,1980-11-03,1591.4
1,1980-11-10,1592.9
2,1980-11-17,1596.3
3,1980-11-24,1597.2
4,1980-12-01,1596.1


In [184]:
dfM2.dtypes

DATE    datetime64[ns]
M2             float64
dtype: object

In [185]:
# Try a join...

dfMerged = pd.merge(dfM2, dfTD, on='DATE', how='left')
dfMerged.tail(5)

,DATE,M2,TD
2096,2021-01-04,19073.0,NaN
2097,2021-01-11,19445.7,NaN
2098,2021-01-18,19462.4,NaN
2099,2021-01-25,19413.7,NaN
2100,2021-02-01,19411.9,NaN


In [186]:
# NaNs for TD because it's reported on Wednesday, M2 is reported on Mondays.
# Subtract two days from TD dates and try again.

dfTD['DATE'] = dfTD['DATE'] - timedelta(days=2)
dfMerged = pd.merge(dfM2, dfTD, on='DATE', how='left')
dfMerged.tail(5)

,DATE,M2,TD
2096,2021-01-04,19073.0,1669.327
2097,2021-01-11,19445.7,1589.421
2098,2021-01-18,19462.4,1626.906
2099,2021-01-25,19413.7,1633.540
2100,2021-02-01,19411.9,1625.325


In [187]:
# Try another join...
dfMerged = pd.merge(dfM2, dfLTD, on='DATE', how='left')
dfMerged.head(20)

,DATE,M2,LTD
0,1980-11-03,1591.4,NaN
1,1980-11-10,1592.9,NaN
2,1980-11-17,1596.3,NaN
3,1980-11-24,1597.2,NaN
4,1980-12-01,1596.1,256.5056
5,1980-12-08,1597.0,NaN
6,1980-12-15,1599.5,NaN
7,1980-12-22,1599.3,NaN
8,1980-12-29,1595.6,NaN
9,1981-01-05,1596.6,NaN


In [188]:
# NaNs for LTD because it's reported monthly (M2 reported on Mondays), there should 
# be around one LTD per month.
# Change LTD dates to 'nearest Monday' dates and try again.
# weekday fxn returns 0-6, where 0 = Monday, 6 = Sunday

def nearest_monday(date):
    weekday = date.weekday()
    if weekday == 0:
        return date
    elif weekday < 4:
        return date + timedelta(days=(0 - weekday))
    else:
        return date + timedelta(days=(7 - weekday))

dfLTD['DATE'] = dfLTD['DATE'].apply(nearest_monday)
dfMerged = pd.merge(dfM2, dfLTD, on='DATE', how='left')
dfMerged.head(20)

,DATE,M2,LTD
0,1980-11-03,1591.4,246.5638
1,1980-11-10,1592.9,NaN
2,1980-11-17,1596.3,NaN
3,1980-11-24,1597.2,NaN
4,1980-12-01,1596.1,256.5056
5,1980-12-08,1597.0,NaN
6,1980-12-15,1599.5,NaN
7,1980-12-22,1599.3,NaN
8,1980-12-29,1595.6,265.8325
9,1981-01-05,1596.6,NaN


In [196]:
# create a list of dataframes, join them all on date
# idea borrowed from Stack Overflow

dfs = [dfM2, dfTC, dfLTD, dfSTD, dfMMF, dfTD]
dfTMS = reduce(lambda left,right: pd.merge(left,right,on='DATE'), dfs)
dfTMS.head(5)

,DATE,M2,TC,LTD,STD,RMMF,TD
0,1986-02-03,2505.7,5.7,350.4186,890.5,175.8,12.475
1,1986-03-03,2521.0,5.7,348.4969,892.8,177.2,4.477
2,1986-03-31,2547.8,5.8,344.7511,896.9,183.4,2.926
3,1986-04-28,2568.9,5.8,339.8018,895.8,187.2,6.769
4,1986-06-02,2594.2,5.8,338.9182,888.9,192.1,2.687


In [198]:
dfTMS.tail(5)

,DATE,M2,TC,LTD,STD,RMMF,TD
391,2018-09-03,14237.8,1.7,1651.1683,515.1,782.9,323.764
392,2018-10-01,14256.8,1.7,1672.3998,527.1,788.1,368.007
393,2018-10-29,14257.8,1.7,1663.8102,534.6,800.2,373.368
394,2018-12-03,14311.8,1.7,1676.2733,548.8,817.1,337.564
395,2018-12-31,14422.7,1.7,1732.4233,559.4,840.6,379.902


In [199]:
# looks decent but was hoping for 1980-11-03 to 2021-02-01 (M2 dates)
# so doing it the non-clever way...

# dfs = [dfM2, dfTC, dfLTD, dfSTD, dfMMF, dfTD]

dfTMS = pd.merge(dfM2, dfTC, on='DATE', how='left')
dfTMS = pd.merge(dfTMS, dfLTD, on='DATE', how='left')
dfTMS = pd.merge(dfTMS, dfSTD, on='DATE', how='left')
dfTMS = pd.merge(dfTMS, dfMMF, on='DATE', how='left')
dfTMS = pd.merge(dfTMS, dfTD, on='DATE', how='left')

dfInputs.head(5)

,DATE,M2,TC,LTD,STD,RMMF,TD
0,1986-01-13,2499.6,5.6,NaN,889.2,172.5,5.032
1,1986-01-20,2503.7,5.6,NaN,889.4,174.3,13.856
2,1986-01-27,2509.1,5.6,NaN,890.4,175.5,17.487
3,1986-02-03,2505.7,5.7,350.4186,890.5,175.8,12.475
4,1986-02-10,2510.6,5.7,NaN,891.1,176.1,6.854


In [200]:
dfTMS.tail(5)

,DATE,M2,TC,LTD,STD,RMMF,TD
1825,2021-01-04,19073.0,NaN,1514.6707,240.5,1059.4,1669.327
1826,2021-01-11,19445.7,NaN,NaN,237.8,1060.2,1589.421
1827,2021-01-18,19462.4,NaN,NaN,230.9,1073.0,1626.906
1828,2021-01-25,19413.7,NaN,NaN,225.7,1071.8,1633.540
1829,2021-02-01,19411.9,NaN,1511.7493,221.1,1075.4,1625.325


In [201]:
# Travelers checks not a big deal but missing treasury deposits probably is.
# TD data begins 1986-01-08, so I'll start there
dfM2 = dfM2[dfM2['DATE'] >= '1986-01-08']
# and try again...
dfTMS = pd.merge(dfM2, dfTC, on='DATE', how='left')
dfTMS = pd.merge(dfTMS, dfLTD, on='DATE', how='left')
dfTMS = pd.merge(dfTMS, dfSTD, on='DATE', how='left')
dfTMS = pd.merge(dfTMS, dfMMF, on='DATE', how='left')
dfTMS = pd.merge(dfTMS, dfTD, on='DATE', how='left')

dfTMS.head(5)


,DATE,M2,TC,LTD,STD,RMMF,TD
0,1986-01-13,2499.6,5.6,NaN,889.2,172.5,5.032
1,1986-01-20,2503.7,5.6,NaN,889.4,174.3,13.856
2,1986-01-27,2509.1,5.6,NaN,890.4,175.5,17.487
3,1986-02-03,2505.7,5.7,350.4186,890.5,175.8,12.475
4,1986-02-10,2510.6,5.7,NaN,891.1,176.1,6.854


In [202]:
dfTMS.tail(5)

,DATE,M2,TC,LTD,STD,RMMF,TD
1825,2021-01-04,19073.0,NaN,1514.6707,240.5,1059.4,1669.327
1826,2021-01-11,19445.7,NaN,NaN,237.8,1060.2,1589.421
1827,2021-01-18,19462.4,NaN,NaN,230.9,1073.0,1626.906
1828,2021-01-25,19413.7,NaN,NaN,225.7,1071.8,1633.540
1829,2021-02-01,19411.9,NaN,1511.7493,221.1,1075.4,1625.325


In [203]:
dfTMS.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1830 entries, 0 to 1829
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   DATE    1830 non-null   datetime64[ns]
 1   M2      1830 non-null   float64       
 2   TC      1721 non-null   float64       
 3   LTD     421 non-null    float64       
 4   STD     1830 non-null   float64       
 5   RMMF    1830 non-null   float64       
 6   TD      1830 non-null   float64       
dtypes: datetime64[ns](1), float64(6)
memory usage: 114.4 KB


In [204]:
# M2 is the reference dataset, so going with 1830 records
# Nulls in TC are due to lack of recent usage so zero those out

dfTMS[['TC']] = dfTMS[['TC']].fillna(value=0)
dfTMS.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1830 entries, 0 to 1829
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   DATE    1830 non-null   datetime64[ns]
 1   M2      1830 non-null   float64       
 2   TC      1830 non-null   float64       
 3   LTD     421 non-null    float64       
 4   STD     1830 non-null   float64       
 5   RMMF    1830 non-null   float64       
 6   TD      1830 non-null   float64       
dtypes: datetime64[ns](1), float64(6)
memory usage: 114.4 KB


In [209]:
# LTD nulls due to it being monthly, so intepolate in some data

dfTMS = dfTMS.interpolate(method='linear', limit_direction='both', axis=0)
dfTMS.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1830 entries, 0 to 1829
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   DATE    1830 non-null   datetime64[ns]
 1   M2      1830 non-null   float64       
 2   TC      1830 non-null   float64       
 3   LTD     1830 non-null   float64       
 4   STD     1830 non-null   float64       
 5   RMMF    1830 non-null   float64       
 6   TD      1830 non-null   float64       
dtypes: datetime64[ns](1), float64(6)
memory usage: 114.4 KB


In [213]:
# Do the needed to calculate TMS

dfTMS['TMS'] = dfTMS.M2 - dfTMS.TC - dfTMS.LTD - dfTMS.STD - dfTMS.RMMF + dfTMS.TD
dfTMS.head(5)

,DATE,M2,TC,LTD,STD,RMMF,TD,TMS
0,1986-01-13,2499.6,5.6,350.418600,889.2,172.5,5.032,1086.913400
1,1986-01-20,2503.7,5.6,350.418600,889.4,174.3,13.856,1097.837400
2,1986-01-27,2509.1,5.6,350.418600,890.4,175.5,17.487,1104.668400
3,1986-02-03,2505.7,5.7,350.418600,890.5,175.8,12.475,1095.756400
4,1986-02-10,2510.6,5.7,349.938175,891.1,176.1,6.854,1094.615825


In [214]:
dfTMS.tail(5)

,DATE,M2,TC,LTD,STD,RMMF,TD,TMS
1825,2021-01-04,19073.0,0.0,1514.67070,240.5,1059.4,1669.327,17927.75630
1826,2021-01-11,19445.7,0.0,1513.94035,237.8,1060.2,1589.421,18223.18065
1827,2021-01-18,19462.4,0.0,1513.21000,230.9,1073.0,1626.906,18272.19600
1828,2021-01-25,19413.7,0.0,1512.47965,225.7,1071.8,1633.540,18237.26035
1829,2021-02-01,19411.9,0.0,1511.74930,221.1,1075.4,1625.325,18228.97570
